In [28]:
# downloading images

from bing_image_downloader import downloader

downloader.download(
    query='terror dark evil shadow',
    output_dir='./images',
    limit=1000,
)

9e394a107b4c.JPG/:/rs=w:365,h:365,cg:true,m/cr=w:365,h:365

[%] Downloading Image #418 from https://thumbs.dreamstime.com/b/senior-couple-sitting-picnic-table-hike-portrait-128588722.jpg
[%] File Downloaded !

[%] Downloading Image #419 from https://thumbs.dreamstime.com/b/summer-weekend-concept-portrait-happy-group-friends-trendy-casual-clothing-drinking-red-wine-talking-having-fun-sunny-149524468.jpg
[%] File Downloaded !

[%] Downloading Image #420 from https://i1.wp.com/www.sillymamaquilts.com/wp-content/uploads/2016/03/DSCN9181.jpg?w=1024
[%] File Downloaded !

[%] Downloading Image #421 from https://i.pinimg.com/originals/7c/9f/62/7c9f625b42412763a8df15e9c813a9e2.jpg
[%] File Downloaded !

[%] Downloading Image #422 from https://i.ebayimg.com/images/g/dKoAAOSw0QFXCx~5/s-l640.jpg
[%] File Downloaded !

[%] Downloading Image #423 from http://thumb1.shutterstock.com/display_pic_with_logo/562825/116828260/stock-photo-young-couple-on-a-romantic-picnic-in-a-park-on-a-sunny-day-outdoors

KeyboardInterrupt: 

In [1]:
# loading downloaded images into 64x64 pixel grayscale images

from PIL import Image
from tqdm import tqdm

def square_image(im):
    (x, y) = im.size
    size = max(x, y)
    squared_im = Image.new('L', (size, size), (0))
    squared_im.paste(
        im,
        box=(
            int((size - x) / 2),
            int((size - y) / 2)
        )
    )
    return squared_im

def process_images(dir_name, pixels=64):
    processed = []
    i = 0
    for imagefile in tqdm(os.listdir(dir_name)):
        im = Image.open(dir_name + imagefile)
        im = im.convert('L')
        im = square_image(im)
        im = im.resize((pixels, pixels), Image.LANCZOS)
        processed.append(im)
        i += 1
    return processed

images = process_images('./images/terror dark evil shadow/')

100%|██████████| 1004/1004 [00:22<00:00, 43.67it/s]


In [5]:
# loading images into numpy and normalizing values from zero to one

import numpy as np
import tensorflow as tf

normalized_images = np.array([np.asarray(im).reshape(64, 64, 1) / 256 for im in images])
random_input = tf.random.normal(normalized_images.shape, mean=0.5, stddev=0.25)

print(normalized_images.shape)
print(random_input.shape)

(1004, 64, 64, 1)
(1004, 64, 64, 1)


In [6]:
import matplotlib.pyplot as plt

def plt_images(images, ncols=5):
    fig = plt.figure(figsize=(40, 40))
    nrows = int(len(images) / ncols) + 1
    for i in range(len(images)):
        plt.subplot(nrows, ncols, i + 1)
        plt.imshow(images[i], cmap='gray')
        plt.axis('off')
    plt.show()

In [7]:
# creating the generator network
# The idea is to transform a random 64x64 image into a scary image

from tensorflow.keras import layers, models, losses, optimizers

generator = models.Sequential()

# Taking a 64-pixel grayscale image as input
# This is intended to be a random normalized image, so the network can learn base images to 'combine'
# Kernel size is meant to be 1/8 the width at any given level
generator.add(layers.Conv2DTranspose(
    filters=64,
    kernel_size=(5, 5),
    strides=(2, 2),
    padding='same',
    use_bias=False,
    input_shape=(64, 64, 1)
))

generator.add(layers.Conv2DTranspose(
    filters=64,
    kernel_size=(5, 5),
    strides=(2, 2),
    padding='same',
    use_bias=False,
))

generator.add(layers.Conv2D(
    filters=64,
    kernel_size=(5, 5),
    strides=(2, 2),
    padding='same',
    use_bias=False,
))

generator.add(layers.Conv2D(
    filters=1,
    kernel_size=(5, 5),
    strides=(2, 2),
    padding='same',
    use_bias=False,
))

print(generator.output_shape)

generator.compile(
    optimizer=optimizers.Adam(),
    loss=losses.MeanSquaredError(),
    metrics=['accuracy'],
)

(None, 64, 64, 1)


In [8]:
# training the model to generate images from a set random array. Overfitting is intentional, as I'm trying to get the model to "learn" these images

generator.fit(
    x=random_input,
    y=normalized_images,
    epochs=100,
)

example_input = tf.random.normal((10, 64, 64, 1), mean=0.5, stddev=0.25)
example_images = generator(example_input, training=False)
plt_images(example_images)

Epoch 1/100


KeyboardInterrupt: 